# 🔍 RAG na Prática: Criando uma IA que Realmente Sabe das Coisas!

## Módulo 10 - LangChain v0.3 Course

Fala, pessoal! Pedro Nunes Guth aqui! 🚀

Bora colocar a mão na massa e implementar nosso primeiro RAG completo! Nos módulos anteriores a gente viu:
- Como fazer document loading e splitting (Módulo 8)
- Como criar embeddings e vector stores (Módulo 9)
- Chains e prompts (Módulos 4 e 6)

Agora é hora de juntar tudo isso numa implementação RAG que vai fazer sua IA parecer que tem PhD em qualquer assunto!

**Tá, mas o que é RAG mesmo?** 🤔

Imagina que você tem um amigo super inteligente, mas que só sabe coisas até 2021. Aí você dá pra ele um monte de livros atualizados e fala: "Ó, antes de responder qualquer coisa, dá uma olhada nesses livros aqui". Isso é RAG!

**RAG = Retrieval Augmented Generation**
- **Retrieval**: Busca informações relevantes
- **Augmented**: Aumenta o conhecimento da IA
- **Generation**: Gera respostas baseadas nessas informações

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-10_img_01.png)

## 📚 Como Funciona o RAG na Prática?

O RAG é tipo aquele sistema de atendimento ao cliente que funciona de verdade (raridade, né?):

1. **Você faz uma pergunta** 🗣️
2. **O sistema busca nos documentos** 🔍
3. **Encontra as partes relevantes** 📄
4. **Monta uma resposta baseada nisso** 🤖

A diferença é que aqui funciona mesmo! 😂

### Fluxo Matemático do RAG:

Para uma query $q$, o RAG funciona assim:

$$\text{resposta} = \text{LLM}(q + \text{retrieve}(q, D))$$

Onde:
- $q$ = sua pergunta
- $D$ = base de documentos
- $\text{retrieve}(q, D)$ = documentos mais relevantes para $q$

### Etapas Detalhadas:

1. **Embedding da Query**: $e_q = \text{embedding}(q)$
2. **Busca por Similaridade**: $\text{sim}(e_q, e_d) = \frac{e_q \cdot e_d}{||e_q|| \cdot ||e_d||}$
3. **Seleção dos Top-k**: $D_{top} = \text{top_k}(\text{sim}(e_q, D))$
4. **Geração**: $\text{resposta} = \text{LLM}(\text{prompt} + D_{top} + q)$

In [ ]:
# Bora instalar as dependências que vamos precisar!
!pip install langchain langchain-google-genai langchain-community faiss-cpu python-dotenv -q

print("Liiindo! Tudo instalado! 🚀")

In [ ]:
# Imports necessários - já conhecemos esses caras dos módulos anteriores!
import os
from dotenv import load_dotenv

# LangChain components que já vimos
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

# Para visualização
import matplotlib.pyplot as plt
import numpy as np

print("Imports feitos! Vamos que vamos! 💪")

In [ ]:
# Configuração da API do Google (lembra do Módulo 2?)
load_dotenv()

# Se você não tem o arquivo .env, descomente e cole sua API key aqui:
# os.environ["GOOGLE_API_KEY"] = "sua_api_key_aqui"

# Verificando se tá tudo certo
if "GOOGLE_API_KEY" not in os.environ:
    print("❌ Opa! Faltou a API key do Google!")
    print("Configure no .env ou descomente a linha acima")
else:
    print("✅ API key configurada! Bora pro RAG!")

## 🏗️ Construindo nosso RAG - Passo 1: Preparando os Documentos

Lembra do Módulo 8? Vamos usar aqueles conceitos de document loading e splitting!

**Dica!** 💡 O splitting é super importante! Se você cortar os documentos muito pequenos, perde contexto. Muito grandes, fica confuso. É tipo cortar um bolo - tem que ser na medida certa!

Vamos criar alguns documentos de exemplo sobre tecnologia brasileira:

In [ ]:
# Criando documentos de exemplo sobre tecnologia brasileira
# (Na vida real, você carregaria de arquivos, APIs, etc.)

documentos_exemplo = [
    """A Nubank foi fundada em 2013 por David Vélez, Cristina Junqueira e Edward Wible. 
    A empresa revolucionou o mercado financeiro brasileiro com seu cartão de crédito roxo 
    e aplicativo mobile. Em 2021, a empresa fez IPO na NYSE e se tornou o banco digital 
    mais valioso da América Latina, com mais de 70 milhões de clientes.""",
    
    """O Pix foi lançado pelo Banco Central do Brasil em novembro de 2020. É um sistema 
    de pagamentos instantâneos que funciona 24 horas por dia, 7 dias por semana. 
    O Pix permite transferências de dinheiro em até 10 segundos e se tornou o meio 
    de pagamento mais popular do Brasil, processando bilhões de transações por mês.""",
    
    """A Stone Pagamentos foi fundada em 2012 e se tornou uma das principais empresas 
    de meios de pagamento do Brasil. A empresa oferece máquinas de cartão (maquininhas), 
    soluções de software e serviços financeiros para pequenas e médias empresas. 
    Está listada na NASDAQ desde 2018.""",
    
    """O iFood é a maior plataforma de delivery de comida da América Latina, fundada em 2011. 
    A empresa conecta restaurantes, entregadores e consumidores através de seu aplicativo. 
    Durante a pandemia de COVID-19, o iFood expandiu significativamente e hoje opera 
    em mais de 1000 cidades brasileiras.""",
    
    """A 99 (anteriormente 99Taxis) foi fundada em 2012 e se tornou a principal 
    concorrente brasileira da Uber. Em 2018, foi adquirida pela chinesa Didi Chuxing 
    por 1 bilhão de dólares. A empresa oferece serviços de transporte por aplicativo 
    e delivery, sendo líder em várias cidades brasileiras."""
]

print(f"Criamos {len(documentos_exemplo)} documentos de exemplo sobre tech brasileira! 🇧🇷")
print(f"Primeiro documento tem {len(documentos_exemplo[0])} caracteres")

In [ ]:
# Transformando em Document objects (padrão do LangChain)
# Lembra disso do Módulo 8?

documents = []
for i, texto in enumerate(documentos_exemplo):
    doc = Document(
        page_content=texto,
        metadata={"source": f"tech_brasileira_doc_{i}", "tipo": "startup"}
    )
    documents.append(doc)

print(f"✅ Convertidos {len(documents)} textos para Document objects!")
print(f"Exemplo de metadata: {documents[0].metadata}")

In [ ]:
# Splitting dos documentos (Módulo 8 feelings!)
# Configurações importantes para um bom RAG

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,        # Tamanho de cada chunk
    chunk_overlap=50,      # Sobreposição entre chunks
    length_function=len,   # Como medir o tamanho
    separators=["\n\n", "\n", ".", " ", ""]  # Como quebrar o texto
)

# Fazendo o split
split_docs = text_splitter.split_documents(documents)

print(f"📄 Documentos originais: {len(documents)}")
print(f"🔪 Chunks após splitting: {len(split_docs)}")
print(f"\n📋 Exemplo de chunk:")
print(f"Conteúdo: {split_docs[0].page_content[:100]}...")
print(f"Metadata: {split_docs[0].metadata}")

## 🧠 Passo 2: Criando Embeddings e Vector Store

Agora vem a parte que vimos no Módulo 9! Vamos transformar nossos textos em vetores numéricos.

**Tá, mas por que vetores?** 🤔

É tipo transformar palavras em coordenadas num mapa gigante. Palavras parecidas ficam perto, palavras diferentes ficam longe. Assim o computador consegue entender "proximidade semântica"!

**Dica!** 💡 O modelo de embedding é crucial! Modelos diferentes vão dar vetores diferentes. No LangChain v0.3, temos várias opções. Aqui vamos usar o Google!

In [ ]:
# Criando o modelo de embeddings (lembra do Módulo 9?)
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

print("🧠 Modelo de embeddings criado!")

# Testando com um texto simples
teste_embedding = embeddings.embed_query("Nubank é um banco digital")
print(f"📊 Dimensão do vetor: {len(teste_embedding)}")
print(f"🔢 Primeiros 5 valores: {teste_embedding[:5]}")

# Cada palavra vira um vetor de 768 números! Liiindo!

In [ ]:
# Criando o Vector Store com FAISS
# FAISS = Facebook AI Similarity Search (agora Meta AI)
# É uma biblioteca super eficiente para busca por similaridade!

print("🏗️ Criando vector store... Isso pode demorar um pouco!")

vector_store = FAISS.from_documents(
    documents=split_docs,
    embedding=embeddings
)

print("✅ Vector store criado com sucesso!")
print(f"📚 Armazenados {len(split_docs)} chunks como vetores")

# Testando uma busca simples
resultados = vector_store.similarity_search("Nubank cartão roxo", k=2)
print(f"\n🔍 Teste de busca por 'Nubank cartão roxo':")
for i, doc in enumerate(resultados):
    print(f"  {i+1}. {doc.page_content[:80]}...")

## 🎯 Visualizando a Similaridade dos Embeddings

Bora ver como fica a similaridade entre diferentes queries e nossos documentos! É tipo ver o "mapa da proximidade semântica".

In [ ]:
# Vamos testar diferentes queries e ver as similaridades
queries_teste = [
    "cartão de crédito roxo",
    "pagamento instantâneo",
    "delivery de comida",
    "transporte por aplicativo",
    "máquina de cartão"
]

# Para cada query, vamos buscar os documentos mais similares
resultados_similaridade = []

for query in queries_teste:
    # Busca com score de similaridade
    docs_com_score = vector_store.similarity_search_with_score(query, k=3)
    
    # Guardando os scores
    scores = [score for _, score in docs_com_score]
    resultados_similaridade.append(scores)
    
    print(f"🔍 Query: '{query}'")
    for i, (doc, score) in enumerate(docs_com_score):
        print(f"  Top {i+1} (score: {score:.3f}): {doc.page_content[:60]}...")
    print()

In [ ]:
# Visualizando os scores de similaridade
fig, ax = plt.subplots(figsize=(12, 8))

# Preparando dados para o gráfico
x_pos = np.arange(len(queries_teste))
width = 0.25

# Plotando os top 3 resultados para cada query
for i in range(3):
    scores_pos = [resultado[i] for resultado in resultados_similaridade]
    ax.bar(x_pos + i*width, scores_pos, width, 
           label=f'Top {i+1}', alpha=0.8)

ax.set_xlabel('Queries de Teste')
ax.set_ylabel('Score de Similaridade (menor = mais similar)')
ax.set_title('📊 Scores de Similaridade por Query\n(Quanto menor, mais similar!)')
ax.set_xticks(x_pos + width)
ax.set_xticklabels(queries_teste, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Lembre-se: no FAISS, scores MENORES indicam MAIOR similaridade!")

## 🤖 Passo 3: Criando o Modelo de Chat

Agora vamos criar nosso modelo de linguagem! Lembra do Módulo 2? Vamos usar o Gemini que já conhecemos bem!

**Dica!** 💡 Para RAG, configurações como temperatura e max_tokens são importantes. Queremos respostas precisas baseadas nos documentos, então temperatura baixa é ideal!

In [ ]:
# Criando o modelo de chat (nosso velho conhecido do Módulo 2!)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # Modelo mais recente
    temperature=0.1,               # Baixa para ser mais preciso
    max_tokens=512                 # Controle do tamanho da resposta
)

print("🤖 Modelo de chat criado!")
print(f"Model: {llm.model}")
print(f"Temperature: {llm.temperature}")
print(f"Max tokens: {llm.max_tokens}")

# Teste rápido
teste_resposta = llm.invoke("Olá! Como você está?")
print(f"\n🧪 Teste: {teste_resposta.content}")

## 📝 Passo 4: Criando um Prompt Template Eficiente

Agora vem a parte crucial! Lembra do Módulo 4 sobre Prompt Templates? Vamos criar um prompt que:
1. Instruir a IA sobre como usar os documentos
2. Dar contexto relevante
3. Fazer a pergunta do usuário

**Dica!** 💡 Um bom prompt de RAG é tipo uma receita de bolo: tem que ter todos os ingredientes na ordem certa!

In [ ]:
# Criando um prompt template específico para RAG
# Isso é arte! Um prompt bem feito faz toda a diferença

rag_prompt_template = """
Você é um assistente especializado em tecnologia e startups brasileiras.

Use APENAS as informações fornecidas no contexto abaixo para responder à pergunta.
Se a informação não estiver no contexto, diga que não tem essa informação disponível.

Seja preciso, objetivo e baseie sua resposta exclusivamente no contexto fornecido.

CONTEXTO:
{context}

PERGUNTA: {question}

RESPOSTA:
"""

# Criando o PromptTemplate (lembra do Módulo 4?)
rag_prompt = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "question"]
)

print("📝 Prompt template para RAG criado!")
print("\n🎯 Variáveis do prompt:", rag_prompt.input_variables)

# Testando o formato do prompt
exemplo_prompt = rag_prompt.format(
    context="Nubank foi fundada em 2013...",
    question="Quando foi fundado o Nubank?"
)
print(f"\n📋 Exemplo de prompt formatado:\n{exemplo_prompt[:200]}...")

## ⚡ Passo 5: Montando a Chain RAG Completa

Agora é a hora da verdade! Vamos juntar tudo que vimos nos módulos anteriores numa chain RAG!

**O que vai rolar:**
1. Recebe a pergunta
2. Busca documentos relevantes no vector store
3. Monta o prompt com contexto + pergunta
4. Envia pro LLM
5. Retorna a resposta baseada nos documentos

É tipo montar um hambúrguer: cada camada tem sua função! 🍔

In [ ]:
# Criando a chain RAG completa!
# Isso aqui é a magia acontecendo - todos os módulos anteriores se juntando!

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,                           # Nosso modelo de chat
    chain_type="stuff",                # Tipo de chain ("stuff" = coloca tudo junto)
    retriever=vector_store.as_retriever(  # Nosso retriever
        search_type="similarity",      # Tipo de busca
        search_kwargs={"k": 3}         # Quantos documentos buscar
    ),
    chain_type_kwargs={
        "prompt": rag_prompt            # Nosso prompt customizado
    },
    return_source_documents=True       # Retorna os documentos usados
)

print("🔗 Chain RAG criada com sucesso!")
print("✅ Componentes conectados:")
print("  - LLM: Gemini 2.0 Flash")
print("  - Vector Store: FAISS")
print("  - Embeddings: Google")
print("  - Prompt: Customizado para RAG")
print("  - Retriever: Top 3 documentos mais similares")

## 🚀 Testando Nosso RAG na Prática!

Chegou a hora da verdade! Vamos fazer perguntas pro nosso RAG e ver se ele consegue responder baseado nos documentos.

**Dica!** 💡 Observe que as respostas vão citar informações específicas dos documentos que carregamos!

In [ ]:
# Primeira pergunta: sobre o Nubank
pergunta1 = "Quando foi fundado o Nubank e quem foram os fundadores?"

print(f"🤔 Pergunta: {pergunta1}")
print("\n🔍 Buscando informações...")

resposta1 = rag_chain({"query": pergunta1})

print(f"\n🤖 Resposta: {resposta1['result']}")
print(f"\n📚 Documentos consultados:")
for i, doc in enumerate(resposta1['source_documents']):
    print(f"  {i+1}. Fonte: {doc.metadata['source']}")
    print(f"     Trecho: {doc.page_content[:80]}...\n")

In [ ]:
# Segunda pergunta: sobre o Pix
pergunta2 = "O que é o Pix e quando foi lançado?"

print(f"🤔 Pergunta: {pergunta2}")
print("\n🔍 Buscando informações...")

resposta2 = rag_chain({"query": pergunta2})

print(f"\n🤖 Resposta: {resposta2['result']}")
print(f"\n📚 Documentos consultados:")
for i, doc in enumerate(resposta2['source_documents']):
    print(f"  {i+1}. {doc.page_content[:100]}...\n")

In [ ]:
# Terceira pergunta: testando os limites
pergunta3 = "Qual é a capital do Japão?"

print(f"🤔 Pergunta: {pergunta3}")
print("\n🔍 Buscando informações...")

resposta3 = rag_chain({"query": pergunta3})

print(f"\n🤖 Resposta: {resposta3['result']}")
print("\n💡 Repare que o RAG não inventa informações que não estão nos documentos!")
print("Isso é uma característica MUITO importante do RAG bem implementado.")

## 📊 Analisando a Performance do RAG

Vamos criar algumas métricas simples para entender como nosso RAG está se saindo!

In [ ]:
# Vamos testar várias perguntas e analisar os resultados
perguntas_teste = [
    "Quando foi fundado o Nubank?",
    "O que é o Pix?", 
    "Quem fundou a Stone?",
    "Em que ano o iFood foi criado?",
    "Quanto a Didi pagou pela 99?",
    "Qual a cor do cartão do Nubank?",
    "Como funciona o delivery do iFood?"
]

resultados_teste = []
tempos_resposta = []

print("🧪 Testando múltiplas perguntas...\n")

import time

for i, pergunta in enumerate(perguntas_teste):
    print(f"  {i+1}/7 - Testando: {pergunta[:40]}...")
    
    # Medindo tempo de resposta
    inicio = time.time()
    resultado = rag_chain({"query": pergunta})
    fim = time.time()
    
    tempo = fim - inicio
    tempos_resposta.append(tempo)
    
    # Analisando a resposta
    resposta = resultado['result']
    num_docs = len(resultado['source_documents'])
    tamanho_resposta = len(resposta)
    
    resultados_teste.append({
        'pergunta': pergunta,
        'resposta': resposta,
        'tempo': tempo,
        'num_docs': num_docs,
        'tamanho': tamanho_resposta
    })

print("\n✅ Testes concluídos!")
print(f"📊 Tempo médio de resposta: {np.mean(tempos_resposta):.2f} segundos")
print(f"📏 Tamanho médio das respostas: {np.mean([r['tamanho'] for r in resultados_teste]):.0f} caracteres")

In [ ]:
# Visualizando a performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tempos de Resposta
ax1.bar(range(len(tempos_resposta)), tempos_resposta, color='skyblue', alpha=0.8)
ax1.set_xlabel('Pergunta #')
ax1.set_ylabel('Tempo (segundos)')
ax1.set_title('⏱️ Tempos de Resposta do RAG')
ax1.grid(True, alpha=0.3)

# Gráfico 2: Tamanho das Respostas
tamanhos = [r['tamanho'] for r in resultados_teste]
ax2.bar(range(len(tamanhos)), tamanhos, color='lightcoral', alpha=0.8)
ax2.set_xlabel('Pergunta #')
ax2.set_ylabel('Caracteres')
ax2.set_title('📏 Tamanho das Respostas')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas
print(f"📈 Estatísticas do RAG:")
print(f"  ⏱️ Tempo médio: {np.mean(tempos_resposta):.2f}s (±{np.std(tempos_resposta):.2f}s)")
print(f"  📏 Resposta média: {np.mean(tamanhos):.0f} chars (±{np.std(tamanhos):.0f} chars)")
print(f"  🚀 Tempo mínimo: {np.min(tempos_resposta):.2f}s")
print(f"  🐌 Tempo máximo: {np.max(tempos_resposta):.2f}s")

## 🔧 Configurações Avançadas do RAG

Agora que temos um RAG funcionando, vamos ver algumas configurações avançadas que podem melhorar muito a performance!

**Configurações importantes:**
- **k (número de documentos)**: Quantos chunks buscar
- **score_threshold**: Filtro de qualidade
- **search_type**: Tipo de busca (similarity, mmr, etc.)

**Dica!** 💡 Nem sempre mais documentos = melhor resposta. Às vezes, poucos documentos relevantes são melhores que muitos medianos!

In [ ]:
# Testando diferentes configurações de k (número de documentos)
ks_para_testar = [1, 2, 3, 5]
pergunta_teste = "O que você sabe sobre o Nubank?"

print(f"🧪 Testando diferentes valores de k com a pergunta: '{pergunta_teste}'\n")

for k in ks_para_testar:
    print(f"📊 Testando k={k}:")
    
    # Criando retriever com k específico
    retriever_k = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )
    
    # Chain temporária com esse k
    chain_temp = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_k,
        chain_type_kwargs={"prompt": rag_prompt},
        return_source_documents=True
    )
    
    resultado = chain_temp({"query": pergunta_teste})
    
    print(f"  📝 Resposta ({len(resultado['result'])} chars): {resultado['result'][:100]}...")
    print(f"  📚 Documentos usados: {len(resultado['source_documents'])}\n")

In [ ]:
# Testando busca com MMR (Maximum Marginal Relevance)
# MMR ajuda a diversificar os resultados, evitando documentos muito similares

print("🔍 Comparando busca por Similarity vs MMR\n")

pergunta_mmr = "Me fale sobre startups de pagamento no Brasil"

# Busca por similaridade normal
retriever_similarity = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Busca com MMR
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "lambda_mult": 0.7  # Balance entre relevância e diversidade
    }
)

# Testando similarity
docs_similarity = retriever_similarity.get_relevant_documents(pergunta_mmr)
print("📊 Busca por Similarity:")
for i, doc in enumerate(docs_similarity):
    print(f"  {i+1}. {doc.page_content[:60]}...")

print("\n🎯 Busca por MMR (mais diversificada):")
docs_mmr = retriever_mmr.get_relevant_documents(pergunta_mmr)
for i, doc in enumerate(docs_mmr):
    print(f"  {i+1}. {doc.page_content[:60]}...")

print("\n💡 MMR tenta trazer documentos relevantes MAS diversos!")

## 🎓 Exercício Prático: Seu Primeiro RAG Customizado

Agora é sua vez! Vamos criar um RAG sobre um tema diferente.

**Desafio:** 🏆
1. Crie documentos sobre **futebol brasileiro** 
2. Configure um RAG com esses documentos
3. Teste perguntas sobre times, jogadores, etc.
4. Compare diferentes configurações de k

**Dica!** 💡 Use o código que já vimos como base. Mude apenas os documentos e teste!

In [ ]:
# SEU CÓDIGO AQUI!
# Crie documentos sobre futebol brasileiro

documentos_futebol = [
    # TODO: Adicione pelo menos 4 documentos sobre:
    # - Times brasileiros
    # - Jogadores famosos 
    # - Copas do Mundo
    # - Campeonatos
    
    """O Pelé, cujo nome real é Edson Arantes do Nascimento, é considerado 
    o maior jogador de futebol de todos os tempos. Nascido em 1940, 
    jogou no Santos FC e na Seleção Brasileira, conquistando 3 Copas do Mundo 
    (1958, 1962, 1970). Marcou mais de 1000 gols na carreira.""",
    
    # Adicione mais documentos aqui...
]

print("⚽ Exercício: Crie seus documentos de futebol aqui!")
print("📝 Dica: Siga o mesmo padrão que usamos para tech brasileira")

# Complete o resto da implementação!

## 📈 Melhorias Avançadas para RAG

Agora que você já domina o básico, vamos ver algumas técnicas avançadas que podem turbinar seu RAG!

### Técnicas Avançadas:

1. **Re-ranking**: Reordenar resultados com modelo adicional
2. **Query Expansion**: Expandir a query original
3. **Hybrid Search**: Combinar busca semântica + keyword
4. **Metadata Filtering**: Filtrar por metadados
5. **Multi-hop RAG**: RAG com múltiplas etapas

**Dica!** 💡 Cada técnica resolve um problema específico. Não use todas de uma vez!

In [ ]:
# Exemplo: Filtros por Metadata
# Vamos buscar apenas documentos de um tipo específico

print("🔍 Testando filtros por metadata...\n")

# Primeiro, vamos ver os metadados disponíveis
print("📋 Metadados disponíveis nos documentos:")
for i, doc in enumerate(split_docs[:3]):
    print(f"  Doc {i+1}: {doc.metadata}")

# Busca com filtro (simulando que temos documentos de tipos diferentes)
# Na vida real, você poderia filtrar por data, autor, categoria, etc.

pergunta_filtro = "Me fale sobre bancos digitais"

# Busca normal (sem filtro)
resultados_normais = vector_store.similarity_search(pergunta_filtro, k=3)

print(f"\n🔍 Busca normal para: '{pergunta_filtro}'")
for i, doc in enumerate(resultados_normais):
    print(f"  {i+1}. Fonte: {doc.metadata.get('source', 'N/A')}")
    print(f"     Conteúdo: {doc.page_content[:80]}...\n")

print("💡 Em projetos reais, você pode filtrar por data, categoria, autor, etc.!")

## 🏗️ Fluxo Completo do RAG - Diagrama

Vamos visualizar todo o processo que implementamos:

In [ ]:
# Criando um diagrama do fluxo RAG
from IPython.display import HTML

diagrama_rag = """
<div class="mermaid">
graph TD
    A[👤 Usuário faz pergunta] --> B[🔍 Embedding da pergunta]
    B --> C[📊 Busca no Vector Store]
    C --> D[📄 Top-k documentos mais similares]
    D --> E[📝 Monta prompt com contexto]
    E --> F[🤖 LLM gera resposta]
    F --> G[✅ Resposta baseada nos documentos]
    
    H[📚 Base de Documentos] --> I[✂️ Text Splitting]
    I --> J[🧠 Embeddings dos chunks]
    J --> K[💾 Armazenamento no FAISS]
    K --> C
    
    style A fill:#e1f5fe
    style G fill:#e8f5e8
    style F fill:#fff3e0
    style C fill:#f3e5f5
</div>
"""

print("🎯 Fluxo Completo do RAG que Implementamos:")
print("")
print("📥 ETAPA 1 - PREPARAÇÃO (offline):")
print("  1. Carregar documentos")
print("  2. Fazer text splitting")
print("  3. Gerar embeddings")
print("  4. Armazenar no vector store")
print("")
print("🔄 ETAPA 2 - CONSULTA (runtime):")
print("  1. Usuário faz pergunta")
print("  2. Embedding da pergunta")
print("  3. Busca similaridade no vector store")
print("  4. Recupera top-k documentos")
print("  5. Monta prompt com contexto")
print("  6. LLM gera resposta")
print("  7. Retorna resposta + fontes")

HTML(diagrama_rag)

## 🚨 Problemas Comuns e Soluções

Todo RAG tem seus desafios. Vamos ver os mais comuns e como resolver:

### Problemas Típicos:

1. **Chunk Size Ruim**: Pedaços muito grandes ou pequenos
2. **Embeddings Inadequados**: Modelo não captura bem o domínio
3. **Prompt Mal Feito**: LLM não entende como usar o contexto
4. **Muitos/Poucos Documentos**: k muito alto/baixo
5. **Qualidade dos Docs**: Documentos com informação ruim

**Dica!** 💡 RAG é mais arte que ciência. Tem que testar e ajustar!

In [ ]:
# Exemplo: Diagnóstico de problemas comuns
print("🩺 Diagnóstico do nosso RAG:\n")

# 1. Análise do chunk size
tamanhos_chunks = [len(doc.page_content) for doc in split_docs]
print(f"📏 Análise dos Chunks:")
print(f"  - Tamanho médio: {np.mean(tamanhos_chunks):.0f} chars")
print(f"  - Menor chunk: {np.min(tamanhos_chunks)} chars")
print(f"  - Maior chunk: {np.max(tamanhos_chunks)} chars")
print(f"  - Total de chunks: {len(split_docs)}")

if np.mean(tamanhos_chunks) < 100:
    print("  ⚠️ AVISO: Chunks muito pequenos podem perder contexto!")
elif np.mean(tamanhos_chunks) > 1000:
    print("  ⚠️ AVISO: Chunks muito grandes podem confundir o LLM!")
else:
    print("  ✅ Tamanho dos chunks parece adequado!")

# 2. Análise da distribuição
print(f"\n📊 Distribuição dos tamanhos:")
hist, bins = np.histogram(tamanhos_chunks, bins=5)
for i in range(len(hist)):
    print(f"  {bins[i]:.0f}-{bins[i+1]:.0f} chars: {hist[i]} chunks")

# 3. Teste de qualidade das buscas
print(f"\n🎯 Teste de qualidade:")
query_teste = "Nubank"
docs_teste = vector_store.similarity_search_with_score(query_teste, k=3)

scores = [score for _, score in docs_teste]
print(f"  - Scores de similaridade: {[f'{s:.3f}' for s in scores]}")
print(f"  - Diferença max-min: {max(scores) - min(scores):.3f}")

if max(scores) - min(scores) < 0.1:
    print("  ⚠️ AVISO: Scores muito próximos - documentos podem ser muito similares!")
else:
    print("  ✅ Boa variação nos scores de similaridade!")

## 🎊 Resumo: O que Aprendemos sobre RAG!

Cara, que jornada! 🚀 Implementamos um RAG completo do zero!

### ✅ O que fizemos:
1. **Carregamos e processamos documentos** (Módulo 8 vibes)
2. **Criamos embeddings e vector store** (Módulo 9 power)
3. **Configuramos LLM e prompts** (Módulos 2, 4, 6 unidos!)
4. **Montamos a chain RAG completa**
5. **Testamos e otimizamos**
6. **Analisamos performance**

### 🧠 Conceitos principais:
- **RAG = Retrieval + Augmented + Generation**
- **Similaridade semântica** com embeddings
- **Chunk size** é crítico
- **Prompt engineering** para RAG
- **k (número de documentos)** afeta qualidade

### 🔮 Próximos passos (Módulo 11):
No próximo módulo vamos ver **Agents e Tools** - imagina um RAG que pode usar ferramentas externas! 🤯

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-10_img_02.png)

In [ ]:
# Teste final: vamos fazer uma pergunta complexa!
pergunta_final = "Compare as diferentes empresas de tecnologia brasileira mencionadas nos documentos"

print(f"🎯 TESTE FINAL: {pergunta_final}\n")

resposta_final = rag_chain({"query": pergunta_final})

print(f"🤖 Resposta Final:")
print(resposta_final['result'])

print(f"\n📚 Documentos consultados: {len(resposta_final['source_documents'])}")
print(f"📊 Tamanho da resposta: {len(resposta_final['result'])} caracteres")

print("\n🎉 Parabéns! Você implementou seu primeiro RAG com LangChain v0.3!")
print("🚀 Agora você pode criar IAs que sabem de qualquer assunto!")
print("📖 Nos próximos módulos vamos ver Agents, Tools e muito mais!")
print("\n💪 Bora continuar essa jornada incrível!")

## 🏋️ Exercício Final: Desafio RAG Completo

**Seu desafio:** 🏆

Implemente um RAG sobre **culinária brasileira** com:
1. Pelo menos 5 documentos sobre pratos típicos
2. Metadata com região (Norte, Sul, etc.)
3. Teste diferentes valores de k
4. Compare similarity vs MMR
5. Crie visualizações dos resultados

**Bonus:** 🌟
- Adicione filtros por região
- Teste diferentes chunk sizes
- Calcule métricas de performance

**Dica!** 💡 Use todo o código que vimos como referência. Você já tem tudo que precisa!

Liiindo! Agora você é um expert em RAG! 🎊